# Coefficients & Residuals

## 0. Meta

In [1294]:
import ctypes
import os
import time
import typing as tp

In [1295]:
import numpy as np
import pandas as pd
from numpy.typing import NDArray

In [1296]:
UInt8s: tp.TypeAlias = NDArray[np.uint8]
Int64s: tp.TypeAlias = NDArray[np.int64]
Floats: tp.TypeAlias = NDArray[np.float64]

In [1297]:
PTR: int = ctypes.sizeof(ctypes.c_void_p)
print(f'{PTR} bytes in a pointer')

8 bytes in a pointer


In [1298]:
MIN: int = -5000
MAX: int = +9999
BPL: int = 12 # bytes per line
LPA: int = 1806 # lines per array

In [1299]:
t0: float = time.perf_counter()
here: str = os.path.abspath('')
csv_dir: str = os.path.join(here, 'build')
lu_path: str = os.path.join(csv_dir, 'lunar.csv')
so_path: str = os.path.join(csv_dir, 'solar.csv')
cpp_dir: str = os.path.join(here, '..', 'fit')
ex_path: str = os.path.join(cpp_dir, 'data.cpp')

In [1300]:
def int_frac(x: float) -> tuple[int, float]:
    '''
    Splits a float to integral and fractional parts.

    Args:
        x (float): input floating-point value
    Returns:
        int: the maximal integer `i <= x`
        float: `x - i` in [0, 1)
    '''

    xi, xf = divmod(x, 1)
    return int(xi), xf

## 1. Lunar

### 1.0. Data

In [1301]:
lu_df: pd.DataFrame = pd.read_csv(lu_path)
lu_va: pd.Series = lu_df['nian'].between(MIN, MAX)
cyue: pd.Series = lu_df['cyue']
lu_df['uday'] = (lu_df['usec'] + 28800) // 86400
uday: pd.Series = lu_df['uday']
lu_df.head()

,cyue,nian,ryue,usec,uday
0,-86209,-5001,22,-219954231530,-2545767
1,-86208,-5001,24,-219951691034,-2545737
2,-86207,-5000,2,-219949157263,-2545708
3,-86206,-5000,4,-219946626852,-2545679
4,-86205,-5000,6,-219944096614,-2545649


### 1.1. N2G: (`nian`, `ryue`) to `cyue` to uday

#### 1.1.1. `nian` to `run`

In [1302]:
nr_ryue: pd.Series = lu_df['ryue']
nr_rn: pd.Series = (lu_df['ryue'] & 1).astype(np.bool)
nr_nian: pd.Series = lu_df['nian'][nr_rn]
nr_ry: pd.Series = nr_ryue[nr_rn] // 2
nr_dc: dict[int, int] = dict(zip(nr_nian, nr_ry))
print(f'{len(nr_dc)} runs')

5708 runs


In [1303]:
NR_BIT, NR_IPB = 4, 2
NR_IPL: int = NR_IPB * BPL
nr_cnt_va: int = (MAX - MIN + 1) + (MIN - MAX - 1) % NR_IPL
nr_ry_bt: list[int] = []
for i in range(0, nr_cnt_va, NR_IPB):
    bt: int = 0
    for j in range(NR_IPB):
        n: int = MIN + i + j
        bt |= nr_dc.get(n, 13) << (j * NR_BIT)
    nr_ry_bt.append(bt)
nr_ry_np: UInt8s = np.array(nr_ry_bt, dtype=np.uint8)
nr_ry_np = nr_ry_np.reshape((-1, BPL))
nr_ry_ln: list[list[int]] = nr_ry_np.tolist()
print(f'{len(nr_ry_bt)} bytes')

7500 bytes


#### 1.1.2. `nian` to `cyue`: linear (1-deg) regression

In [1304]:
ny_va: pd.Series = lu_df['ryue'] == 2
ny_nian: pd.Series = lu_df['nian'][ny_va]
ny_dp01: pd.Series = lu_df['cyue'][ny_va]
ny_coef: Floats = np.polyfit(x=ny_nian, y=ny_dp01, deg=1)
ny_c1, ny_c0 = tp.cast(list[float], ny_coef.tolist())
print(f'cyue = {ny_c1} * nian + {ny_c0}')

cyue = 12.368273905932472 * nian + -24365.550975329075


In [1305]:
ny_c1i, ny_c1f = int_frac(ny_c1)
ny_c0i, ny_c0f = int_frac(ny_c0)
print(ny_c1i, ny_c1f, ny_c0i, ny_c0f)

12 0.36827390593247245 -24366 0.44902467092470033


In [1306]:
NY_BIT, NY_IPB = 1, 8
for ny_bits in range(32):
    ny_c0ib: int = ny_c0i
    ny_c1bi, _ = int_frac(ny_c1f * (1 << ny_bits))
    ny_c0bi, _ = int_frac(ny_c0f * (1 << ny_bits))
    ny_pred: pd.Series = ny_c1i * ny_nian + ny_c0i
    ny_bias: pd.Series = ny_c1bi * ny_nian + ny_c0bi
    ny_pred += ny_bias // (1 << ny_bits)
    ny_resd: pd.Series = ny_dp01 - ny_pred
    if ny_resd.max() - ny_resd.min() < (1 << NY_BIT):
        ny_c0ib += ny_resd.min()
        ny_resd -= ny_resd.min()
        print(f'cyue = {ny_c1i} * nian + {ny_c0ib}', end=' + ')
        print(f'({ny_c1bi} * nian + {ny_c0bi} >> {ny_bits})')
        print(f'resd in {range(ny_resd.min(), ny_resd.max() + 1)}')
        break
else:
    raise ValueError('bad idea, residuals too wide')

cyue = 12 * nian + -24366 + (6033 * nian + 7356 >> 14)
resd in range(0, 2)


In [1307]:
NY_IPL: int = NY_IPB * BPL
ny_va: pd.Series = ny_nian.between(MIN, MAX)
ny_cnt_va: int = ny_va.sum() + 1 + (-(ny_va.sum() + 1) % NY_IPL)
ny_resd_va: pd.Series = ny_resd[ny_nian >= MIN].iloc[:ny_cnt_va]
ny_resd_bt: list[int] = []
for i in range(0, len(ny_resd_va), NY_IPB):
    bt: int = 0
    for j, r in enumerate(ny_resd_va.iloc[i:i+NY_IPB]):
        bt |= r << (j * NY_BIT)
    ny_resd_bt.append(bt)
ny_resd_np: UInt8s = np.array(ny_resd_bt, dtype=np.uint8)
ny_resd_np = ny_resd_np.reshape((-1, BPL))
ny_resd_ln: list[list[int]] = ny_resd_np.tolist()
print(f'{len(ny_resd_bt)} bytes')

1884 bytes


#### 1.1.3. `cyue` to uday: linear (1-deg) regression

In [1308]:
yd_coef: Floats = np.polyfit(x=cyue, y=uday, deg=1)
yd_c1, yd_c0 = tp.cast(list[float], yd_coef.tolist())
print(f'uday = {yd_c1} * cyue + {yd_c0}')

uday = 29.53058513341781 * cyue + 35.7110938177826


In [1309]:
yd_c1i, yd_c1f = int_frac(yd_c1)
yd_c0i, yd_c0f = int_frac(yd_c0)
print(yd_c1i, yd_c1f, yd_c0i, yd_c0f)

29 0.5305851334178087 35 0.7110938177825972


In [1310]:
YD_BIT, YD_IPB = 2, 4
for yd_bits in range(64):
    yd_c0ib: int = yd_c0i
    yd_c1bi, _ = int_frac(yd_c1f * (1 << yd_bits))
    yd_c0bi, _ = int_frac(yd_c0f * (1 << yd_bits))
    yd_pred: pd.Series = yd_c1i * cyue + yd_c0i
    yd_bias: pd.Series = yd_c1bi * cyue + yd_c0bi
    yd_pred += yd_bias // (1 << yd_bits)
    yd_resd: pd.Series = uday - yd_pred
    if yd_resd.max() - yd_resd.min() < 1 << YD_BIT:
        yd_c0ib += yd_resd.min()
        yd_resd -= yd_resd.min()
        print(f'uday = {yd_c1i} * cyue + {yd_c0ib}', end=' + ')
        print(f'({yd_c1bi} * cyue + {yd_c0bi} >> {yd_bits})')
        print(f'resd in {range(yd_resd.min(), yd_resd.max() + 1)}')
        break
else:
    raise ValueError('bad idea, residuals too wide')

uday = 29 * cyue + 34 + (278179 * cyue + 372817 >> 19)
resd in range(0, 4)


In [1311]:
YD_IPL: int = YD_IPB * BPL
yd_cnt_va: int = lu_va.sum() + 1 + (-(lu_va.sum() + 1) % YD_IPL)
yd_va: pd.Series = lu_df['nian'] >= MIN
yd_resd_va: pd.Series = yd_resd[yd_va].iloc[:yd_cnt_va]
yd_resd_bt: list[int] = []
for i in range(0, len(yd_resd_va), YD_IPB):
    bt: int = 0
    for j, r in enumerate(yd_resd_va.iloc[i:i+YD_IPB]):
        bt |= r << (j * YD_BIT)
    yd_resd_bt.append(bt)
yd_resd_np: UInt8s = np.array(yd_resd_bt, dtype=np.uint8)
yd_resd_np = yd_resd_np.reshape((-1, BPL))
yd_resd_ln: list[list[int]] = []
for i in range(0, yd_resd_np.shape[0], LPA):
    yd_resd_ln.append(yd_resd_np[i:i+LPA].tolist())
print(f'{len(yd_resd_bt)} bytes')

46392 bytes


### 1.2. G2N: uday to `cyue` to (`nian`, `ryue`)

#### 1.2.1. uday to `cyue`: linear (1-deg) regression

In [1312]:
dy_coef: Floats = np.polyfit(x=uday, y=cyue, deg=1)
dy_c1, dy_c0 = tp.cast(list[float], dy_coef.tolist())
dy_c0 += 0.5 # it is midpoint that is to be predicted
print(f'cyue = {dy_c1} * cday + {dy_c0}')

cyue = 0.03386319625845294 * cday + -0.7092917774042613


In [1313]:
_, dy_c1f = int_frac(dy_c1)
dy_c0i, dy_c0f = int_frac(dy_c0)
print(dy_c1f, dy_c0i, dy_c0f)

0.03386319625845294 -1 0.2907082225957387


In [1314]:
for dy_bits in range(64):
    dy_c1bi, _ = int_frac(dy_c1f * (1 << dy_bits))
    dy_c0bi, _ = int_frac(dy_c0f * (1 << dy_bits))
    dy_bias: pd.Series = dy_c1bi * uday + dy_c0bi
    dy_pred: pd.Series = dy_c0i + dy_bias // (1 << dy_bits)
    if np.all(dy_pred == cyue):
        print(f'cyue = {dy_c0i}', end=' + ')
        print(f'({dy_c1bi} * uday + {dy_c0bi} >> {dy_bits})')
        break
else:
    raise ValueError('bad idea, residuals too wide')

cyue = -1 + (8877 * uday + 76207 >> 18)


#### 1.2.2. `cyue` to `nian`: linear (1-deg) regression

In [1315]:
yn_cyue: pd.Series = lu_df['cyue'][lu_df['ryue'] == 2]
yn_nian: pd.Series = lu_df['nian'][lu_df['ryue'] == 2]
yn_coef: Floats = np.polyfit(x=yn_cyue, y=yn_nian, deg=1)
yn_c1, yn_c0 = tp.cast(list[float], yn_coef.tolist())
yn_c0 += 0.5 # similarly, midpoints are considered
print(f'nian = {yn_c1} * cyue + {yn_c0}')

nian = 0.0808520257214669 * cyue + 1970.5041542503538


In [1316]:
_, yn_c1f = int_frac(yn_c1)
yn_c0i, yn_c0f = int_frac(yn_c0)
print(yn_c1f, yn_c0i, yn_c0f)

0.0808520257214669 1970 0.5041542503538494


In [1317]:
for yn_bits in range(64):
    yn_c1bi, _ = int_frac(yn_c1f * (1 << yn_bits))
    yn_c0bi, _ = int_frac(yn_c0f * (1 << yn_bits))
    yn_bias: pd.Series = yn_c1bi * yn_cyue + yn_c0bi
    yn_pred: pd.Series = yn_c0i + yn_bias // (1 << yn_bits)
    if np.all(yn_pred == yn_nian):
        print(f'nian = {yn_c0i}', end=' + ')
        print(f'({yn_c1bi} * cyue + {yn_c0bi} >> {yn_bits})')
        break
else:
    raise ValueError('bad idea, residuals too wide')

nian = 1970 + (10597 * cyue + 66080 >> 17)


## 2. Solar

### 2.0. Data

In [1318]:
so_df: pd.DataFrame = pd.read_csv(so_path)
cjie: pd.Series = so_df['cjie']
suis: pd.Series = so_df['sui']
jies: pd.Series = so_df['jie']
usec: pd.Series = so_df['usec']
so_va: pd.Series = suis.between(MIN, MAX)
so_df.head()

,cjie,sui,jie,usec
0,-167280,-5000,0,-219953114960
1,-167279,-5000,1,-219951781386
2,-167278,-5000,2,-219950436354
3,-167277,-5000,3,-219949081820
4,-167276,-5000,4,-219947720330


### 2.1. `sui` to usec: sextic (6-deg) regression

In [1319]:
ss_coef_ls: list[Floats] = []
for jie in range(24):
    ss_gr_suis: pd.Series = suis[jies == jie]
    ss_gr_usec: pd.Series = usec[jies == jie]
    ss_coef_gr: Floats = np.polyfit(x=ss_gr_suis, y=ss_gr_usec, deg=6)
    ss_coef_ls.append(ss_coef_gr)
ss_coef_np: Floats = np.array(ss_coef_ls, dtype=np.float64)
ss_c0_np: Floats = ss_coef_np[:, -1]
ss_c0m, _ = int_frac(ss_c0_np.min().item())
ss_coef_np[:, -1] -= ss_c0m
ss_c1_np: Floats = ss_coef_np[:, -2]
ss_c1m, _ = int_frac(ss_c1_np.min().item())
ss_coef_np[:, -2] -= ss_c1m
print(f'coefficient matrix shape {ss_coef_np.shape}')

coefficient matrix shape (24, 7)


In [1320]:
SS_BIT: int = 12
ss_va: pd.Series = suis >= MIN
for ss_bits in range(32):
    ss_shls: Floats = 2.0 ** (ss_bits * np.arange(6, -1, -1))
    ss_cshl_np: Floats = ss_coef_np * ss_shls
    ss_ci_np: Int64s = np.round(ss_cshl_np).astype(np.int64)
    ss_rest_ls: list[list[int]] = []
    ss_ress_ext_ls: list[int] = []
    for jie in range(24):
        ss_gr_suis: pd.Series = suis[jies == jie]
        ss_gr_usec: pd.Series = usec[jies == jie]
        ss_gr_c0, *su_gr_c1r = tp.cast(list[int], ss_ci_np[jie].tolist())
        ss_gr_pr: pd.Series = 0 * ss_gr_suis + ss_gr_c0
        for ci in su_gr_c1r:
            ss_gr_pr = ss_gr_pr * ss_gr_suis // (1 << ss_bits) + ci
        ss_gr_pr += ss_c1m * ss_gr_suis + ss_c0m
        ss_gr_ress = ss_gr_usec - ss_gr_pr
        ss_rest_ls.append(ss_gr_ress[ss_va].to_list())
        ss_ress_ext_ls.append(ss_gr_ress.max() - ss_gr_ress.min())
    if max(ss_ress_ext_ls) < 1 << SS_BIT:
        ss_resm_ls: list[list[int]] = list(zip(*ss_rest_ls))
        ss_resm_np: Int64s = np.array(ss_resm_ls, dtype=np.int64)
        ss_resm_min: Int64s = ss_resm_np.min(axis=0)
        ss_ci_np[:, -1] += 2 * (1 << SS_BIT) + ss_resm_min
        ss_c0mb: int = ss_c0m - 2 * (1 << SS_BIT)
        print(f'usec = {ss_c1m} * sui + {ss_c0mb} + ({ss_bits} bits/deg)')
        ss_resm_np -= ss_resm_min
        print(f'resd in {range(ss_resm_np.min(), ss_resm_np.max() + 1)}')
        break
else:
    raise ValueError('bad idea, residuals too wide')

usec = 31556894 * sui + -62168131570 + (12 bits/deg)
resd in range(0, 3127)


In [1321]:
SS_IPL: int = SS_BIT * 8 // BPL
ss_cnt_va: int = so_va.sum() + 1 + (-so_va.sum() - 1) % SS_IPL
ss_ress_va: list[int] = ss_resm_np.ravel()[:ss_cnt_va].tolist()
ss_ress_bt: list[int] = []
# with SU_BIT == 12 == 1.5 bytes
for old, new in zip(ss_ress_va[0::2], ss_ress_va[1::2]):
    lo: int = old & 0xff
    md: int = (old >> 8) | (new & 0x0f)
    hi: int = new >> 4
    ss_ress_bt.extend([lo, md, hi])
ss_ress_np: UInt8s = np.array(ss_ress_bt, dtype=np.uint8)
ss_ress_np = ss_ress_np.reshape((-1, BPL))
ss_ress_ln: list[list[int]] = []
for i in range(0, ss_ress_np.shape[0], LPA):
    ss_ress_ln.append(ss_ress_np[i:i+LPA].tolist())
print(f'{len(ss_ress_bt)} bytes')

540012 bytes


### 2.2. usec to `cjie`: linear (1-deg) regression

In [1322]:
sj_coef: Floats = np.polyfit(x=usec, y=cjie, deg=1)
sj_c1, sj_c0 = tp.cast(list[float], sj_coef.tolist())
sj_c0 += 0.5
print(f'cjie = {sj_c1} * usec + {sj_c0}')

cjie = 7.605305803404133e-07 * usec + 1.2641291881802954


In [1323]:
_, sj_c1f = int_frac(sj_c1)
sj_c0i, sj_c0f = int_frac(sj_c0)
print(sj_c1f, sj_c0i, sj_c0f)

7.605305803404133e-07 1 0.2641291881802954


In [1324]:
for sj_bits in range(32):
    sj_c1bi, _ = int_frac(sj_c1f * (1 << (2 * sj_bits)))
    sj_c0bi, _ = int_frac(sj_c0f * (1 << sj_bits))
    sj_pred: pd.Series = (sj_c1bi * usec) // (1 << sj_bits)
    sj_pred = (sj_pred + sj_c0bi) // (1 << sj_bits) + sj_c0i
    if np.all(sj_pred == cjie):
        print(f'cjie = {sj_c0i}', end=' + ')
        print(f'(({sj_c1bi} * usec >> {sj_bits})', end=' + ')
        print(f'{sj_c0bi} >> {sj_bits})')
        break
else:
    raise ValueError('bad idea, residuals too wide')

cjie = 1 + ((209053 * usec >> 19) + 138479 >> 19)


## 3. Export

### 3.0. Formats

In [1325]:
VPL: int = 5 # variables per line
ICX: str = 'constexpr inline'
with open(ex_path, 'w') as ex_cpp:
    pass # clear `data.cpp` if it exists
ex_fmt_bt: str = '   ' + BPL * ' 0x{:02x},' + '\n'
print(ex_fmt_bt[:60])

    0x{:02x}, 0x{:02x}, 0x{:02x}, 0x{:02x}, 0x{:02x}, 0x{:02


In [1326]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write('#include <cstdint>\n\n')
    ex_cpp.write(f'{ICX} int64_t NIAN_MIN = {MIN:+d};\n')
    ex_cpp.write(f'{ICX} int64_t NIAN_MAX = {MAX:+d};\n')
    ex_cpp.write(f'{ICX} int64_t NIAN_NUM = {MAX - MIN + 1};\n\n')
    ex_cpp.write(f'{ICX} int64_t SUI_MIN = {MIN:+d};\n')
    ex_cpp.write(f'{ICX} int64_t SUI_MAX = {MAX:+d};\n')
    ex_cpp.write(f'{ICX} int64_t SUI_NUM = {MAX - MIN + 1};\n\n')
    ex_cpp.write(f'{ICX} int64_t YEAR_MIN = {MIN:+d};\n')
    ex_cpp.write(f'{ICX} int64_t YEAR_MAX = {MAX:+d};\n')
    ex_cpp.write(f'{ICX} int64_t YEAR_NUM = {MAX - MIN + 1};\n\n')
ex_bt_meta: int = 8 * 9
print(f'{ex_bt_meta} bytes for meta')

72 bytes for meta


### 3.1. Lunar

#### 3.1.0. Formats

In [1327]:
ex_fl_hi: str = ', '.join(4 * ['{:d}'])
ex_fl_wl: list[int] = np.char.str_len(np.array([
    [dy_c0i, dy_c1bi, dy_c0bi],
    [yn_c0i, yn_c1bi, yn_c0bi],
], dtype=str)).max(axis=0).tolist()
ex_fl_lo: str = ', '.join(3 * ['{{:{}d}}']).format(*ex_fl_wl)
print(ex_fl_hi)
print(ex_fl_lo)

{:d}, {:d}, {:d}, {:d}
{:4d}, {:5d}, {:5d}


#### 3.1.1. `nian` to `run`

In [1328]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} uint8_t NR_RUNS[] = {{\n')
    for ex_nr_ln in nr_ry_ln:
        ex_cpp.write(ex_fmt_bt.format(*ex_nr_ln))
    ex_cpp.write('};\n\n')
ex_bt_nr: int = 1 * len(nr_ry_bt)
print(f'{ex_bt_nr} bytes for NR runs')

7500 bytes for NR runs


#### 3.1.2. `nian` to `cyue`

In [1329]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t NY_BITS = {ny_bits};\n\n')
ex_bt_ny_bits: int = 8 # 1 saves nothing due to alignment
print(f'{ex_bt_ny_bits} bytes for NY bits')

8 bytes for NY bits


In [1330]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t NY_COEF[] = {{ ')
    ex_cpp.write(ex_fl_hi.format(ny_c1i, ny_c0ib, ny_c1bi, ny_c0bi))
    ex_cpp.write(' };\n\n')
ex_bt_ny_coef: int = 8 * 4
print(f'{ex_bt_ny_coef} bytes for NY coef')

32 bytes for NY coef


In [1331]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} uint8_t NY_RESD[] = {{\n')
    for ex_ny_ln in ny_resd_ln:
        ex_cpp.write(ex_fmt_bt.format(*ex_ny_ln))
    ex_cpp.write('};\n\n')
ex_bt_ny_resd: int = 1 * len(ny_resd_bt)
print(f'{ex_bt_ny_resd} bytes for NY resd')

1884 bytes for NY resd


#### 3.1.3. `cyue` to uday

In [1332]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t YD_BITS = {yd_bits};\n\n')
ex_bt_yd_bits: int = 8
print(f'{ex_bt_yd_bits} bytes for YD bits')

8 bytes for YD bits


In [1333]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t YD_COEF[] = {{ ')
    ex_cpp.write(ex_fl_hi.format(yd_c1i, yd_c0ib, yd_c1bi, yd_c0bi))
    ex_cpp.write(' };\n\n')
ex_bt_yd_coef: int = 8 * 4
print(f'{ex_bt_yd_coef} bytes for YD coef')

32 bytes for YD coef


In [1334]:
with open(ex_path, 'a') as ex_cpp:
    ex_yd_na: int = len(yd_resd_ln)
    ex_yd_nw: int = len(str(ex_yd_na))
    ex_yd_ns: list[str] = [
        f'YD_RESD_{i:0{ex_yd_nw}d}' for i in range(ex_yd_na)
    ]
    for ex_yd_nv, ex_yd_ap in zip(ex_yd_ns, yd_resd_ln):
        ex_cpp.write(f'{ICX} uint8_t {ex_yd_nv}[] = {{\n')
        for ex_yd_ln in ex_yd_ap:
            ex_cpp.write(ex_fmt_bt.format(*ex_yd_ln))
        ex_cpp.write('};\n\n')
    ex_cpp.write(f'{ICX} const uint8_t *const YD_ARRD[] = {{\n')
    for i in range(0, ex_yd_na, VPL):
        ex_yd_al: list[str] = ex_yd_ns[i:i+VPL]
        ex_yd_ll: int = len(ex_yd_al)
        ex_yd_fmt: str = '   ' + ex_yd_ll * ' {},' + '\n'
        ex_cpp.write(ex_yd_fmt.format(*ex_yd_al))
    ex_cpp.write('};\n\n')
ex_bt_yd_resd: int = 1 * len(yd_resd_bt) + PTR * ex_yd_na
print(f'{ex_bt_yd_resd} bytes for YD resd')

46416 bytes for YD resd


#### 3.1.4. uday to `cyue`

In [1335]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t DY_BITS = {dy_bits};\n\n')
ex_bt_dy_bits: int = 8
print(f'{ex_bt_dy_bits} bytes for DY bits')

8 bytes for DY bits


In [1336]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t DY_COEF[] = {{ ')
    ex_cpp.write(ex_fl_lo.format(dy_c0i, dy_c1bi, dy_c0bi))
    ex_cpp.write(' };\n\n')
ex_bt_dy_coef: int = 8 * 3
print(f'{ex_bt_dy_coef} bytes for DY coef')

24 bytes for DY coef


#### 3.1.5. `cyue` to `nian`

In [1337]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t YN_BITS = {yn_bits};\n\n')
ex_bt_yn_bits: int = 8
print(f'{ex_bt_yn_bits} bytes for YN bits')

8 bytes for YN bits


In [1338]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t YN_COEF[] = {{ ')
    ex_cpp.write(ex_fl_lo.format(yn_c0i, yn_c1bi, yn_c0bi))
    ex_cpp.write(' };\n\n')
ex_bt_yn_coef: int = 8 * 3
print(f'{ex_bt_yn_coef} bytes for YN coef')

24 bytes for YN coef


### 3.2. Solar

#### 3.2.0. Formats

In [1339]:
ex_fs_wh: Int64s = np.char.str_len(ss_ci_np.astype(str)).max(axis=0)
ex_fs_hi: str = ', '.join(7 * ['{{:{}d}}']).format(*ex_fs_wh)
ex_fs_lo: str = ', '.join(3 * ['{:d}'])
print(ex_fs_hi)
print(ex_fs_lo)

{:4d}, {:5d}, {:6d}, {:6d}, {:7d}, {:6d}, {:8d}
{:d}, {:d}, {:d}


#### 3.2.1. `sui` to `usec`

In [1340]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t SS_BITS = {ss_bits};\n\n')
ex_bt_ss_bits: int = 8
print(f'{ex_bt_ss_bits} bytes for SS bits')

8 bytes for SS bits


In [1341]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t SS_COEF[][7] = {{\n')
    for ex_ss_ci in tp.cast(list[int], ss_ci_np.tolist()):
        ex_ss_ci_ln: str = ex_fs_hi.format(*ex_ss_ci)
        ex_cpp.write(f'    {{ {ex_ss_ci_ln} }},\n')
    ex_cpp.write('};\n\n')
ex_bt_ss_coef: int = 8 * 7 * 24
print(f'{ex_bt_ss_coef} bytes for SS coef')

1344 bytes for SS coef


In [1342]:
with open(ex_path, 'a') as ex_cpp:
    ex_ss_na: int = len(ss_ress_ln)
    ex_ss_nw: int = len(str(ex_ss_na))
    ex_ss_ns: list[str] = [
        f'SS_RESS_{i:0{ex_ss_nw}d}' for i in range(ex_ss_na)
    ]
    for ex_ss_nv, ex_ss_ap in zip(ex_ss_ns, ss_ress_ln):
        ex_cpp.write(f'{ICX} uint8_t {ex_ss_nv}[] = {{\n')
        for ex_ss_ln in ex_ss_ap:
            ex_cpp.write(ex_fmt_bt.format(*ex_ss_ln))
        ex_cpp.write('};\n\n')
    ex_cpp.write(f'{ICX} const uint8_t *const SS_ARRS[] = {{\n')
    for i in range(0, ex_ss_na, VPL):
        ex_ss_al: list[str] = ex_ss_ns[i:i+VPL]
        ex_ss_ll: int = len(ex_ss_al)
        ex_ss_fmt: str = '   ' + ex_ss_ll * ' {},' + '\n'
        ex_cpp.write(ex_ss_fmt.format(*ex_ss_al))
    ex_cpp.write('};\n\n')
ex_bt_ss_resd: int = 1 * len(ss_ress_bt) + PTR * ex_ss_na
print(f'{ex_bt_ss_resd} bytes for SS ress')

540212 bytes for SS ress


#### 3.2.2. `usec` to `cjie`

In [1343]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t SJ_BITS = {sj_bits};\n\n')
ex_bt_sj_bits: int = 8
print(f'{ex_bt_sj_bits} bytes for SJ bits')

8 bytes for SJ bits


In [1344]:
with open(ex_path, 'a') as ex_cpp:
    ex_cpp.write(f'{ICX} int64_t SJ_COEF[] = {{ ')
    ex_cpp.write(ex_fs_lo.format(sj_c0i, sj_c1bi, sj_c0bi))
    ex_cpp.write(' };\n\n')
ex_bt_sj_coef: int = 8 * 3
print(f'{ex_bt_sj_coef} bytes for SJ coef')

24 bytes for SJ coef


## 4. Statistics

In [1345]:
ex_bt: int = sum([eval(v) for v in dir() if v.startswith('ex_bt_')])
print(f'{ex_bt} bytes exported as data')
taken: float = time.perf_counter() - t0
print(f'{taken:.3f} s taken for data generation')

597612 bytes exported as data
2.741 s taken for data generation


## 5. Vocabulary

Lunar:
* NR: `nian` to `run`
* NY: `nian` to `cyue`
* YD: `cyue` to `uday`
* DY: `uday` to `cyue`
* YN: `cyue` to `nian`

Solar:
* SS: `sui` to `usec`
* SJ: `usec` to `cjie`

P.S.
* A new `nongli` day comes at 00:00:00 UTC+8,
  and DST from 1985 to 1991 is taken out of account.
* 1970 `nian` comes on `chunjie` (p01-01), while
  1970 `sui` comes at `dongzhi` (1969-12-22 08:43:41 UTC+8).
* The prefix `u` in `uday` and `usec` means Unix,
  counting from Unix Epoch (1970-01-01 00:00:00 UTC).
* The prefix `c` in `cyue` and `cjie` means cumulative,
  counting from 1970-p01 and 1970 `dongzhi` resp.
* `Nongli` has both lunar and solar. NEVER refer to
  CHINESE new year `chunjie` as LUNAR new year!